In [2]:
import pandas as pd
import numpy as np

# 1. Chargement du fichier brut
# Note : Assure-toi que le chemin '../DATA/RAW/...' est correct par rapport à l'emplacement de ton notebook
df = pd.read_csv('../DATA/RAW/dirty_cafe_sales.csv')

# 2. Standardisation : On transforme les textes 'ERROR'/'UNKNOWN' en vraies valeurs vides (NaN)
df.replace(['ERROR', 'UNKNOWN'], np.nan, inplace=True)

# 3. Conversion des types (Indispensable pour que la logique des prix fonctionne)
# 'errors=coerce' transforme les valeurs non convertibles en NaN
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

# --- DÉBUT DE LA LOGIQUE DE CORRECTION OPTIMISÉE (RAPIDE) ---

print(f"Items manquants avant correction : {df['Item'].isna().sum()}")

# Définition des règles métier (Prix Uniques seulement)
mapping_correction = {
    1.0: 'Cookie',
    1.5: 'Tea',
    2.0: 'Coffee',
    5.0: 'Salad'
}

# L'ASTUCE RAPIDE (Vectorisation) :
# 1. On crée une série temporaire avec les noms déduits grâce au prix
items_deduits = df['Price Per Unit'].map(mapping_correction)

# 2. On remplit les trous dans 'Item' avec ces valeurs déduites
# fillna ne touche PAS aux lignes qui ont déjà un nom, il ne remplit que les vides
df['Item'] = df['Item'].fillna(items_deduits)

print(f"Items manquants après correction : {df['Item'].isna().sum()}")

# Correction inverse : Remplir les PRIX manquants si on a l'ITEM
# On peut le faire pour tous les items car chaque item a un prix unique
prix_officiels = {
    'Coffee': 2.0, 'Tea': 1.5, 'Sandwich': 4.0, 'Salad': 5.0,
    'Cake': 3.0, 'Cookie': 1.0, 'Smoothie': 4.0, 'Juice': 3.0
}
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Item'].map(prix_officiels))

# Recalcul du Total Spent pour garantir la cohérence mathématique
df['Total Spent'] = df['Quantity'] * df['Price Per Unit']

# --- FIN DE LA LOGIQUE DE CORRECTION ---

# 4. Nettoyage final
# On remplit les infos non critiques par "Unknown"
df['Payment Method'] = df['Payment Method'].fillna('Unknown')
df['Location'] = df['Location'].fillna('Unknown')

# On supprime seulement les lignes irrécupérables (ex: pas de date, ou pas d'item ET pas de prix)
df_clean = df.dropna(subset=['Transaction Date', 'Item', 'Price Per Unit', 'Quantity'])

# 5. Sauvegarde
df_clean.to_csv('../DATA/PROCESSED/cleaned_cafe_sales.csv', index=False)
print("Fichier nettoyé et sauvegardé dans DATA/PROCESSED/cleaned_cafe_sales.csv !")

Items manquants avant correction : 969
Items manquants après correction : 501
Fichier nettoyé et sauvegardé dans DATA/PROCESSED/cleaned_cafe_sales.csv !


In [1]:
import os
print("Dossier actuel :", os.getcwd())

# Vérifie si le fichier existe bien là où on pense
import os.path
chemin_fichier = '../DATA/RAW/dirty_cafe_sales.csv'
print(f"Est-ce que le fichier existe ici '{chemin_fichier}' ? :", os.path.exists(chemin_fichier))

Dossier actuel : c:\Users\doria\Downloads\DATA-REFINEMENT-PROJECT\NOTEBOOKS
Est-ce que le fichier existe ici '../DATA/RAW/dirty_cafe_sales.csv' ? : True


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../DATA/RAW/dirty_cafe_sales.csv')

# 0. Standardisation : Tout ce qui est texte "bizarre" devient NaN
df.replace(['ERROR', 'UNKNOWN'], np.nan, inplace=True)

# 1. STRATÉGIE "VALEUR PAR DÉFAUT" (Pour Location & Payment)
# On ne supprime rien, on étiquette "Unknown"
cols_cat = ['Location', 'Payment Method']
for col in cols_cat:
    df[col] = df[col].fillna('Unknown')

# 2. STRATÉGIE "LOGIQUE MÉTIER" (Pour Item & Prix)
# On convertit en numérique d'abord
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')

# Dictionnaire officiel
mapping_prix = {1.0: 'Cookie', 1.5: 'Tea', 2.0: 'Coffee', 5.0: 'Salad'}
mapping_item = {'Cookie': 1.0, 'Tea': 1.5, 'Coffee': 2.0, 'Salad': 5.0, 
                'Cake': 3.0, 'Sandwich': 4.0, 'Smoothie': 4.0, 'Juice': 3.0}

# A. Si Item manque, on devine grâce au Prix
df['Item'] = df['Item'].fillna(df['Price Per Unit'].map(mapping_prix))

# B. Si Prix manque, on devine grâce à l'Item
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Item'].map(mapping_item))

# 3. STRATÉGIE "MATHÉMATIQUE" (Pour Quantité et Total)
# A. Si Quantité manque (mais qu'on a Total et Prix) -> Qty = Total / Prix
mask_qty = df['Quantity'].isna() & df['Total Spent'].notna() & df['Price Per Unit'].notna()
df.loc[mask_qty, 'Quantity'] = df.loc[mask_qty, 'Total Spent'] / df.loc[mask_qty, 'Price Per Unit']

# B. Si Total manque (ou est faux), on le recalcule -> Total = Qty * Prix
# Cela écrase les erreurs potentielles dans Total Spent
df['Total Spent'] = df['Quantity'] * df['Price Per Unit']

# 4. STRATÉGIE "TEMPORELLE" (Pour la Date)
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
# On trie par index (optionnel) et on remplit les trous avec la date de la ligne d'avant
df['Transaction Date'] = df['Transaction Date'].ffill() 



# 1. Pour les Items ambigus (les 501 restants), on met "Unknown"
# On ne supprime pas la ligne car le chiffre d'affaire (Total Spent) existe !
df['Item'] = df['Item'].fillna('Unknown Item')

# 2. Pour les quelques lignes sans Prix ni Quantité (les ~50 restantes)
# On les supprime car elles ne servent à rien pour l'analyse (0$ de vente)
df_final = df.dropna(subset=['Quantity', 'Price Per Unit', 'Total Spent'])

# VÉRIFICATION ULTIME
print("--- Check Final (Doit afficher 0 partout) ---")
print(df_final.isnull().sum())

# SAUVEGARDE FINALE
df_final.to_csv('../DATA/PROCESSED/cleaned_cafe_sales.csv', index=False)
print(f"Dataset propre sauvegardé avec {len(df_final)} lignes restantes.")



--- Check Final (Doit afficher 0 partout) ---
Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64
Dataset propre sauvegardé avec 9926 lignes restantes.
